<a target="_parent" href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/demo/navigator/data-designer-101/the-basics.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🎨 Data Designer 101: The Basics

In this notebook, we will demonstrate the basics of `DataDesigner` by generating a simple product review dataset.

## 💾 Install `gretel-client` and its dependencies

In [1]:
%%capture
%pip install git+https://github.com/gretelai/gretel-python-client ipywidgets

In [ ]:
from gretel_client.navigator_client import Gretel

import gretel_client.data_designer.params as P
import gretel_client.data_designer.columns as C

# The Gretel object is the SDK's main entry point for interacting with Gretel's API.
gretel = Gretel(api_key="prompt", endpoint="https://api.dev.gretel.ai")

## ⚙️ Initialize Data Designer with a Model Suite

- `DataDesigner` groups LLMs into "Model Suites" based on the permissiveness of the model licenses.


- Example Model Suites include "apache-2.0" and "llama-3.x".

In [2]:
# Initialize a new Data Designer instance using the `data_designer` factory.
aidd = gretel.data_designer.new(model_suite="apache-2.0")

## 🎲 Getting started with sampler columns


- Sampler columns offer non-LLM based generation of synthetic data. 

- They are particularly useful for **steering the diversity** of the generated data, as we demonstrate below.


<br>

Let's start designing our product review dataset by adding product category and subcategory columns.

In [ ]:
aidd.add_column(
    C.SamplerColumn(
        name="product_category", 
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=["Electronics", "Clothing", "Home & Kitchen", "Books", "Toys"], 
        )
    )
)

aidd.add_column(
    C.SamplerColumn(
        name="product_subcategory",
        type=P.SamplerType.SUBCATEGORY,
        params=P.SubcategorySamplerParams(
            category="product_category",  
            values={
                "Electronics": ["Smartphones", "Laptops", "Headphones", "Cameras", "Accessories"],
                "Clothing": ["Menswear", "Womenswear", "Winter Coats", "Activewear", "Accessories"],
                "Home & Kitchen": ["Appliances", "Cookware", "Furniture", "Decor", "Organization"],
                "Books": ["Fiction", "Non-Fiction", "Self-Help", "Textbooks", "Classics"],
                "Toys": ["Action Figures", "Board Games", "Educational", "Outdoor", "Plush"]
            }
        )
    )
)


aidd.add_column(
    C.SamplerColumn(
        name="target_age_range",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=["18-25", "25-35", "35-50", "50-65", "65+"]
        )
    )
)

# Optionally validate that the columns are configured correctly.
aidd.validate()


Next, let's add samplers to generate data related to the customer and their review.

In [ ]:
# This column will sample synthetic person data based on statistics from the US Census.
aidd.add_column(
    C.SamplerColumn(
        name="customer",
        type=P.SamplerType.PERSON,
        params=P.PersonSamplerParams(age_range=[18, 70])
    )
)

aidd.add_column(
    C.SamplerColumn(
        name="number_of_stars",
        type=P.SamplerType.UNIFORM,
        params=P.UniformSamplerParams(low=1, high=5),
        convert_to="int"
    )
)

aidd.add_column(
    C.SamplerColumn(
        name="review_style",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=["rambling", "brief", "detailed", "structured with bullet points"],
            weights=[1, 2, 2, 1]
        )
    )
)

aidd.validate()

## 🦜 LLM-generated columns

- The real power of `DataDesigner` comes from leveraging LLMs to generate text, code, and structured data.

- For this our product review dataset, we will use LLM-generated text and structured data columns.

- When prompting the LLM, we can use Jinja templates to reference other columns in the dataset.

- As we will see with the person sampler for our `customer` column, nested json columns can be accessed using dot notation.

In [ ]:
aidd.add_column(
    C.LLMTextColumn(
        name="product_name",
        prompt=(
            "Come up with a creative product name for a product in the '{{ product_category }}' category, focusing "
            "on products related to '{{ product_subcategory }}' products. The target age range of the customer is "
            "{{ target_age_range }} years old. Respond with only the product name, no other text."
        ),
        # This is optional, but it can be useful for controlling the behavior of the LLM. Do not include instructions
        # related to output formatting in the system prompt, as AIDD handles this based on the column type.
        system_prompt=(
            "You are a helpful assistant that generates product names. You respond with only the product name, "
            "no other text. You do NOT add quotes around the product name. "
        )
    )
)

aidd.add_column(
    C.LLMTextColumn(
        name="customer_review",
        prompt=(
            "You are a customer named {{ customer.first_name }} from {{ customer.city }}, {{ customer.state }}. "
            "You recently purchased product called {{ product_name }}. Write a review of this product, which "
            "you gave a rating of {{ number_of_stars }} stars. The style of the review should be '{{ review_style }}'. "
        ),
    )
)

aidd.validate()

## 👀 Preview the dataset

- Iteration is key to generating high-quality synthetic data.

- Use the `preview` method to generate 10 records for inspection.

In [ ]:
preview = aidd.preview()

In [ ]:
# Run this cell multiple times to cycle through the 10 preview records.
preview.display_sample_record()

## 🆙 Scale up!

- Once you are happy with the preview, scale up to a larger dataset by submitting a batch workflow

In [ ]:
workflow_run = aidd.create(num_records=100, name="aidd-101-product-reviews")

## ⏭️ Next Steps

Now that you've seen the basics of `DataDesigner`, check out the following notebooks to learn more about:

- Structured outputs and complex jinja expressions

- Seeding synthetic data with your real data

- Use case: text-to-python and text-to-sql code generation

- Use case: RAG evaluation datasets 